In [1]:
import pandas as pd

In [2]:
# This is just an extract of the bookings data set
data = pd.read_csv('testdata.csv', sep=',', header=0)

In [3]:
len(data)

16726

In [4]:
data.columns

Index(['Booking date', 'Quantity', 'Customer ID', 'Product ID', 'Channel',
       'Landingpage', 'Channel_Name'],
      dtype='object')

In [5]:
data.head()

,Booking date,Quantity,Customer ID,Product ID,Channel,Landingpage,Channel_Name
0,01.01.2016,2,1155,204,4,8,Store
1,01.01.2016,2,3367,204,4,8,Store
2,01.01.2016,2,4825,497,1,1,Affiliate A
3,01.01.2016,2,95,157,4,8,Store
4,01.01.2018,2,6212,138,0,0,CRM


In [6]:
# sort the data so we can efficiently compare line n with line n+1
data.sort_values(by=['Customer ID', 'Booking date'], axis=0, ascending=True, inplace=True)

In [7]:
# Is this the customers 1st second, third, ... booking?
data['Booking Sequence'] = data.groupby(['Customer ID']).cumcount()+1;

In [8]:
data.tail(20)

,Booking date,Quantity,Customer ID,Product ID,Channel,Landingpage,Channel_Name,Booking Sequence
582,02.03.2017,1,9988,330,0,0,CRM,1
3173,07.01.2017,2,9988,63,0,0,CRM,2
7354,14.03.2017,1,9988,164,4,6,Store,3
8657,16.08.2015,2,9988,91,3,3,SEA,4
9087,17.03.2017,1,9988,263,0,0,CRM,5
9590,18.01.2019,2,9989,211,2,2,Direct,1
9754,18.06.2018,2,9990,426,0,0,CRM,1
13675,25.11.2017,2,9990,540,5,19,Affiliate B,2
585,02.03.2017,2,9991,298,4,6,Store,1
7498,14.08.2017,1,9992,178,4,7,Store,1


In [9]:
data['Booking Sequence'].describe()

count    16726.000000
mean         1.886285
std          1.614095
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         21.000000
Name: Booking Sequence, dtype: float64

In [10]:
data.reset_index(inplace=True)
len(data)

16726

In [11]:
## unperformant solution, but describes what we want to do: On which channel was the customer aquired last time?
#for i in range(1, len(data)):
#    if (groupdata.loc[i, 'Customer ID'] == groupdata.loc[i-1, 'Customer ID']):
#        #print(str(data.loc[i, 'Customer ID']) + '==' + str(data.loc[i-1, 'Customer ID']))
#        data.loc[i, 'Previous Channel_Name'] = data.loc[i-1, 'Channel_Name']
#        data.loc[i, 'Previous Channel'] = data.loc[i-1, 'Channel']
#        data.loc[i, 'Previous Product ID'] = data.loc[i-1, 'Product ID']
#        data.loc[i, 'Previous Landingpage'] = data.loc[i-1, 'Landingpage']

In [12]:
# performant solution: shift the index by 1 and join again to original df
#Customer ID 	Product ID 	Channel 	Landingpage 	Channel_Name 	Booking Sequence
data_shifted = data[['Customer ID', 'Product ID', 'Channel', 
                     'Channel_Name', 'Landingpage', 'Booking Sequence', 'Booking date']].copy(deep=True)
data_shifted.index = data_shifted.index+1
data = data.join(data_shifted, how='left', rsuffix='_shifted')

In [13]:
data.columns

Index(['index', 'Booking date', 'Quantity', 'Customer ID', 'Product ID',
       'Channel', 'Landingpage', 'Channel_Name', 'Booking Sequence',
       'Customer ID_shifted', 'Product ID_shifted', 'Channel_shifted',
       'Channel_Name_shifted', 'Landingpage_shifted',
       'Booking Sequence_shifted', 'Booking date_shifted'],
      dtype='object')

In [14]:
data[(data['Customer ID'] == data['Customer ID_shifted'])].head(20)

,index,Booking date,Quantity,Customer ID,Product ID,Channel,Landingpage,Channel_Name,Booking Sequence,Customer ID_shifted,Product ID_shifted,Channel_shifted,Channel_Name_shifted,Landingpage_shifted,Booking Sequence_shifted,Booking date_shifted
2,12175,22.12.2017,2,1,207,0,0,CRM,2,1.0,211.0,0.0,CRM,0.0,1.0,18.12.2018
6,13031,24.09.2018,2,4,119,3,3,SEA,2,4.0,16.0,4.0,Store,7.0,1.0,17.05.2017
7,15429,29.01.2019,2,4,38,0,0,CRM,3,4.0,119.0,3.0,SEA,3.0,2.0,24.09.2018
9,6412,12.09.2018,1,5,10,4,6,Store,2,5.0,141.0,5.0,Affiliate B,18.0,1.0,02.10.2017
14,6228,12.04.2019,1,9,424,0,0,CRM,2,9.0,201.0,0.0,CRM,0.0,1.0,04.07.2018
15,9135,17.05.2017,1,9,284,0,0,CRM,3,9.0,424.0,0.0,CRM,0.0,2.0,12.04.2019
16,15061,28.08.2015,1,9,241,4,8,Store,4,9.0,284.0,0.0,CRM,0.0,3.0,17.05.2017
17,16036,30.05.2016,1,9,425,0,0,CRM,5,9.0,241.0,4.0,Store,8.0,4.0,28.08.2015
18,16288,30.10.2018,1,9,309,0,0,CRM,6,9.0,425.0,0.0,CRM,0.0,5.0,30.05.2016
22,5696,11.06.2018,2,12,152,3,3,SEA,2,12.0,355.0,0.0,CRM,0.0,1.0,10.08.2018


In [15]:
# 'Previous Booking date', 'Previous Product ID','Previous Channel', 'Previous Landingpage', 'Previous Channel_Name',

data['Previous Booking date'] = pd.NaT
data.loc[data['Customer ID'] == data['Customer ID_shifted'],'Previous Booking date']= data['Booking date_shifted']


In [16]:
data['Previous Product ID'] = None
data.loc[data['Customer ID'] == data['Customer ID_shifted'],'Previous Product ID']= data['Product ID_shifted']


In [17]:
data['Previous Channel'] = None
data.loc[data['Customer ID'] == data['Customer ID_shifted'],'Previous Channel']= data['Channel_shifted']


In [18]:
data['Previous Channel_Name'] = None
data.loc[data['Customer ID'] == data['Customer ID_shifted'],'Previous Channel_Name']= data['Channel_Name_shifted']


In [19]:
data['Previous Landingpage'] = None
data.loc[data['Customer ID'] == data['Customer ID_shifted'],'Previous Landingpage']= data['Landingpage_shifted']


In [20]:
data_save= data.copy(deep=True)

In [21]:
data=data[['Booking date', 'Quantity', 'Customer ID', 'Product ID',
       'Channel', 'Landingpage', 'Channel_Name', 'Booking Sequence',
       'Previous Booking date', 'Previous Product ID', 'Previous Channel',
       'Previous Channel_Name', 'Previous Landingpage']]

In [22]:
data.head(20)

,Booking date,Quantity,Customer ID,Product ID,Channel,Landingpage,Channel_Name,Booking Sequence,Previous Booking date,Previous Product ID,Previous Channel,Previous Channel_Name,Previous Landingpage
0,17.10.2015,2,0,50,3,3,SEA,1,NaT,None,None,None,None
1,18.12.2018,2,1,211,0,0,CRM,1,NaT,None,None,None,None
2,22.12.2017,2,1,207,0,0,CRM,2,18.12.2018,211,0,CRM,0
3,27.09.2018,2,2,152,3,4,SEA,1,NaT,None,None,None,None
4,12.11.2015,1,3,525,2,2,Direct,1,NaT,None,None,None,None
5,17.05.2017,2,4,16,4,7,Store,1,NaT,None,None,None,None
6,24.09.2018,2,4,119,3,3,SEA,2,17.05.2017,16,4,Store,7
7,29.01.2019,2,4,38,0,0,CRM,3,24.09.2018,119,3,SEA,3
8,02.10.2017,1,5,141,5,18,Affiliate B,1,NaT,None,None,None,None
9,12.09.2018,1,5,10,4,6,Store,2,02.10.2017,141,5,Affiliate B,18


In [23]:
data.dtypes

Booking date             object
Quantity                  int64
Customer ID               int64
Product ID                int64
Channel                   int64
Landingpage               int64
Channel_Name             object
Booking Sequence          int64
Previous Booking date    object
Previous Product ID      object
Previous Channel         object
Previous Channel_Name    object
Previous Landingpage     object
dtype: object

In [24]:
data.to_csv('ChannelMigration.csv')